# Use the OPTIMADE Gateway on the MarketPlace platform

This notebook outlines how one can perform OPTIMADE searches through the OPTIMADE Gateway available on [the MarketPlace platform](https://the-marketplace.eu).

In [ ]:
import os

os.environ["OPTIMADE_CONFIG_FILE"] = "../../config_heroku.yml"

from enum import Enum
import json
from optimade.adapters import Structure
from optimade_gateway.models import QueriesResponseSingle
from pydantic import ValidationError
from random import randint
import requests
from time import sleep

## MarketPlace

Using the OPTIMADE Gateway from the MarketPlace.

Changes:
- Go through the proxy service, using capabilities.
- Provide an authorization token.

In [ ]:
MARKETPLACE_OPTIMADE_APP = (
    "https://staging.the-marketplace.eu"
    "/api/proxy/proxy/1bb6b264-d896-47e2-b54d-6d49e05299f2"
)

class Capabilities(Enum):
    """Implemented capabilities by the OPTIMADE Gateway."""

    HEARTBEAT = "heartbeat"  # GET /heartbeat
    GET_COLLECTION = "getCollection"  # GET /queries/<ID>
    QUERY_COLLECTION = "queryCollection"  # GET /search
    POST_QUERY_COLLECTION = "postQueryCollection"  # POST /search

Now:

1. Follow [this guide](https://marketplace.pages.fraunhofer.de/platform/documentation/application-provider-docs/access_token.html) to obtain an access token.
1. Paste in the token in the code cell below.
1. "Purchase" the "OPTIMADE Gateway" application from [the MarketPlace App Store](https://staging.the-marketplace.eu/appstore).

In [ ]:
# Paste in your authentication token between the quotations (`"`) here:
USER_AUTH_TOKEN = ""

response = requests.get(
    f"{MARKETPLACE_OPTIMADE_APP}/{Capabilities.HEARTBEAT.value}",
    headers={"Authorization": f"Bearer {USER_AUTH_TOKEN}"},
)
if response.ok:
    print(response.text)
else:
    raise RuntimeError(f"Response:\n{response.text}")

### Perform OPTIMADE search through the MarketPlace

In [ ]:
database_ids = [
    "mcloud/sssp",
    "cod",
    "mp",
    "mcloud/mc3d-structures",
]

optimade_filter = 'elements HAS ALL "Si","Al","Mg"'

In [ ]:
response = requests.post(  # POST /search
    f"{MARKETPLACE_OPTIMADE_APP}/{Capabilities.POST_QUERY_COLLECTION.value}",
    json={
        "database_ids": database_ids,
        "query_parameters": {"filter": optimade_filter},
    },
    headers={"Authorization": f"Bearer {USER_AUTH_TOKEN}"},
)
if response.ok:
    search_response = QueriesResponseSingle(**response.json())
else:
    try:
        parsed_response = json.dumps(response.json(), indent=2)
        if "_gateway_traceback" in response.json().get("meta", {}):
            parsed_response += (
                f"\nTraceback:\n{response.json()['meta']['_gateway_traceback']}"
            )
    except json.JSONDecodeError:
        parsed_response = response.text
    raise RuntimeError(
        f"Unsuccessful response.\nStatus code: {response.status_code}\n"
        f"Response:\n{parsed_response}"
    )

state = search_response.data.attributes.state
while state != state.FINISHED:
    sleep(2)
    response = requests.get(  # GET /queries/<ID>
        f"{MARKETPLACE_OPTIMADE_APP}/{Capabilities.GET_COLLECTION.value}",
        params={"query_id": search_response.data.id},
        headers={"Authorization": f"Bearer {USER_AUTH_TOKEN}"},
    )
    if response.ok:
        query_response = QueriesResponseSingle(**response.json())
        state = query_response.data.attributes.state
    else:
        try:
            parsed_response = json.dumps(response.json(), indent=2)
            if "_gateway_traceback" in response.json().get("meta", {}):
                parsed_response += (
                    f"\nTraceback:\n{response.json()['meta']['_gateway_traceback']}"
                )
        except json.JSONDecodeError:
            parsed_response = response.text
        raise RuntimeError(
            f"Unsuccessful response.\nStatus code: {response.status_code}\n"
            f"Response:\n{parsed_response}"
        )

print("Query overview:")
print(
    "\n".join(
        f"- {id_} (Returned results: {len(data)})"
        for id_, data in query_response.data.attributes.response.data.items()
    )
)

### Curate and process search results

In [ ]:
structures = {}
for (
    database, database_structures
) in query_response.data.attributes.response.data.items():
    structures[database] = []
    for structure in database_structures:
        try:
            structures[database].append(Structure(structure.dict()))
        except ValidationError:
            continue

print("\nQuery overview (after validating structure properties):")
print(
    "\n".join(
        f"- {id_} (Valid results: {len(data)})"
        for id_, data in structures.items()
    )
)

In [ ]:
for db_structures in structures.values():
    if db_structures:
        a_structure = db_structures[randint(0, len(db_structures)-1)]
print(a_structure.as_pdb)